# Variables de Kemna-Vorst : options asiatiques

In [1]:
import numpy as np
import pandas as pd
from numpy.random import default_rng, SeedSequence
import scipy.stats as sps
import matplotlib.pyplot as plt
from scipy.stats import norm
import warnings

sq = SeedSequence()
seed = sq.entropy
rng = default_rng(sq)

### Outils : générateur de browniens, de trajectoires BS, formule de payoff, formules fermées de BS

In [2]:
def brownian_1d(n_times: int, n_paths: int, 
                final_time: float=1.0, 
                increments: bool=False, 
                random_state: np.random.Generator=rng) -> np.array:
    """Simulate paths of standard Brownian motion
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths 
        final_time: Final time of simulation
        increments: If `True` the increments of the paths are returned.
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths if the argument `increments` is `False`
        `np.array` of shape `(n_times, n_paths)` containing the increments if the argument `increments` is `True`
    """
    dB = np.sqrt(final_time / n_times) * random_state.standard_normal((n_times, n_paths))
    if increments:
        return dB
    else:
        brownian = np.zeros((n_times+1, n_paths))
        brownian[1:] = np.cumsum(dB, axis=0)
        return brownian

In [3]:
def black_scholes_1d(n_times: int, n_paths: int, 
                     final_time: float=1.0, 
                     random_state: np.random.Generator=rng, *,
                     init_value: float,
                     r: float, sigma: float) -> np.array:
    """Simulate paths of Black-Scholes process
    Args:
        n_times: Number of timesteps
        n_paths: Number of paths 
        final_time: Final time of simulation
        init_value: `S0`
        r: Interest rate
        sigma: Volatility
        random_state: `np.random.Generator` used for simulation
    Returns:
        `np.array` of shape `(n_times+1, n_paths)` containing the paths 
    """
    Bt = brownian_1d(n_times, n_paths)
    times = np.arange(n_times+1)*(1/n_times)
    t = times[:, np.newaxis]
    St = init_value * np.exp((r - 0.5*sigma**2)*t + sigma*Bt)
    return St

In [4]:
# une fonction BS pour un payoff qui n'est pas path-dependent (offre plus de liberté pour le choix des gaussiennes dans la fonction)
def BS(x,r,sigma,T,N):
    """ args :
            x=spot
            r=interest rate
            sigma=volatility
            T=maturity
            N=simulated standard normal random variable
    """
    return  x*np.exp((r-(sigma**2)/2)*T+sigma*np.sqrt(T)*N)

# une fonction de payoff du call et sa dérivée par rapport à S_T
def payoff_call(S,r,T,K): return np.exp(-r*T)*np.maximum(S-K,0)
def payoff_put(S,r,T,K): return np.exp(-r*T)*np.maximum(K-S,0)

def call_derive(S,r,T,K): return np.exp(-r*T)*np.where(S>K,1,0)

In [5]:
def monte_carlo(sample, proba = 0.95):
    mean = np.mean(sample)
    var = np.var(sample, ddof=1)
    alpha = 1 - proba 
    quantile = norm.ppf(1 - alpha/2)  # fonction quantile 
    ci_size = quantile * np.sqrt(var / sample.size)
    return (mean, var, mean - ci_size, mean + ci_size)

In [6]:
# Les formules fermées de Black-Scholes pour vérifier nos méthodes de MC

def d1(spot, t, r, sigma, strike):
    return (np.log(spot / strike) + t * (r + 0.5*sigma**2)) / (sigma * np.sqrt(t))

def d2(spot, t, r, sigma, strike):
    return d1(spot, t, r, sigma, strike) - sigma * np.sqrt(t)

def price_call_BS(spot, t, r, sigma, strike):
    d1_ = d1(spot, t, r, sigma, strike)
    d2_ = d2(spot, t, r, sigma, strike)
    return spot * norm.cdf(d1_) - strike * np.exp(-r * t) * norm.cdf(d2_)

def delta_BS(spot, t, r, sigma, strike):
    d1_ = d1(spot, t, r, sigma, strike)
    return norm.cdf(d1_)

### Pour tester nos fonctions : Pricing MC standard

In [7]:
# Fixons les paramètres

S0 = 100
T=1
K=100
r, sigma = 0.04, 0.20

In [8]:
Ms = 10**np.arange(3, 8)
results = pd.DataFrame(index=['Mean', 'Var', 'Lower', 'Upper'], columns=Ms)
for M in Ms:
    gaussiennes = rng.standard_normal(M)
    payoffs=payoff_call(BS(x=S0,r=r,sigma=sigma,T=T,N=gaussiennes),r=r,T=T,K=K)
    results[M] = monte_carlo(payoffs)
results

,1000,10000,100000,1000000,10000000
Mean,9.712122,10.042327,9.886496,9.926891,9.926359
Var,189.080850,207.154298,206.833680,208.259954,207.870333
Lower,8.859863,9.760233,9.797359,9.898607,9.917422
Upper,10.564382,10.324422,9.975634,9.955176,9.935295


In [9]:
price_call_BS(S0, T, r, sigma, K)

9.925053717274437

Méthode de Monte Carlo standard :
$$
 e^{-rT} \mathbb E(\phi (I))
$$
Méthode de Kemna-Vorst :
$$
 e^{-rT}\mathbb E(\phi (I) - k^{KV})+ Premium(xe^{-(\frac{r}{2}+\frac{\sigma^2}{12})T},r,\frac{\sigma}{\sqrt3},T)
$$

## Pricing option asiatique sans réduction de variance

In [29]:
t=500
T=0.5
r=0.02
K=97
S0=112
sigma=0.2

Ms = 10**np.arange(3, 6)
results = pd.DataFrame(index=['Mean', 'Var', 'Lower', 'Upper'], columns=Ms)
for M in Ms:
    I=np.mean(black_scholes_1d(n_times=t,n_paths=M,final_time=T,init_value=S0,r=r,sigma=sigma),axis=0)
    payoffs=payoff_call(I,r=r,T=T,K=K)
    results[M] = monte_carlo(payoffs)
results

,1000,10000,100000
Mean,16.608182,16.482610,16.435522
Var,144.889252,152.064239,150.956510
Lower,15.862135,16.240918,16.359371
Upper,17.354229,16.724301,16.511672


## Méthode Kemna-Vorst

Comme précédemment, on calcule d'abord la prime.

In [30]:
# Calcule de la prime
premium=price_call_BS(spot=S0*np.exp(-((r/2)+(sigma**2)/12)*T), t=T, r=r, sigma=sigma/np.sqrt(3), strike=K)
premium

15.340826261694772

On calcule maintenant l'espérance à laquelle on ajoute la prime.

In [31]:
# une fonction qui renvoie une trajectoire BS pour des Browniens déjà simulés, pour plus de flexibilité

def BS_browniens(browniens:np.array,n_times: int,
                     final_time: float, 
                     init_value: float,
                     r: float, sigma: float) -> np.array:
    
    Bt = browniens
    times = np.arange(n_times+1)*(1/n_times)
    t = times[:, np.newaxis]
    St = init_value * np.exp((r - 0.5*sigma**2)*t + sigma*Bt)
    return St

In [32]:
def kVT(phi,r,T,K, browniens):
    return phi(S0*np.exp(-((r/2)+(sigma**2)/12)*T)*np.exp((r-(sigma**2)/6)*T+sigma*np.mean(browniens)),r,T,K)

In [33]:
def asim(browniens,n_times,
                     final_time, 
                     init_value: float,
                     r: float, sigma: float):
    return (payoff_call(BS_browniens(browniens, n_times, final_time, init_value, r, sigma),r,T,K)-np.exp(-r*T)*kVT(payoff_call,r,T,K,browniens))*np.exp(-r*T)

In [34]:
# Note : les browniens simulés dans kVT doit être les mêmes que ceux du modèle BS dans l'espérance.

Ms = 10**np.arange(3, 6)
results = pd.DataFrame(index=['Mean', 'Var', 'Lower', 'Upper'], columns=Ms)
for M in Ms:
    MB=brownian_1d(n_times=t,n_paths=M,final_time=T)
    payoffs=asim(MB,n_times=t,final_time=T,init_value=S0,r=r,sigma=sigma)
    results[M] = monte_carlo(payoffs)
    results[M].iloc[0]+=premium
    results[M].iloc[2:]+=premium
results

,1000,10000,100000
Mean,16.324788,16.420358,16.409622
Var,101.990380,108.828718,107.979857
Lower,16.296823,16.411223,16.406744
Upper,16.352753,16.429492,16.412499


On constate une variance significativement réduite.